# Plot results for siloed analyses

In this notebook we review and explore the *All of Us* data for lipids phenotypes and covariates and GWAS results.

# Setup

In [ ]:
lapply(c('hrbrthemes', 'skimr', 'qqman'),
       function(pkg) { if(! pkg %in% installed.packages()) { 
           install.packages(pkg)
       } } )

In [ ]:
library(grid)
library(gridExtra)
library(hrbrthemes)
library(qqman)
library(readxl)
library(scales)
library(skimr)
library(tidyverse)

In [ ]:
# Set some visualiation defaults.
theme_set(theme_ipsum(base_size = 16)) # Default theme for plots.

#' Returns a data frame with a y position and a label, for use annotating ggplot boxplots.
#'
#' @param d A data frame.
#' @return A data frame with column y as max and column label as length.
get_boxplot_fun_data <- function(df) {
  return(data.frame(y = max(df), label = stringr::str_c('N = ', length(df))))
}

# Constants

In [ ]:
# Inputs
REGENIE_PHENO <- 'gs://PATH/TO/aou_alpha2_lipids_phenotypes.tsv'

REGENIE_RESULTS <- c(
    HDL='gs://PATH/TO/aou_alpha2_lipids_regenie_step2_HDL_mg_dl_norm.regenie',
    LDL='gs://PATH/TO/aou_alpha2_lipids_regenie_step2_LDL_adj_mg_dl_norm.regenie',
    TC='gs://PATH/TO/aou_alpha2_lipids_regenie_step2_TC_adj_mg_dl_norm.regenie',
    TG='gs://PATH/TO/aou_alpha2_lipids_regenie_step2_TG_log_mg_dl_norm.regenie'
)

LIPIDS <- names(REGENIE_RESULTS)

LD_PRUNED_VARIANTS <- 'gs://PATH/TO/aou_alpha2plink_ld.prune.in'

PLOT_SUBTITLE <- 'Source: All of Us genomics alpha2 release'

# Load the phenotypes

In [ ]:
pheno = read_tsv(pipe(str_glue('gsutil cat {REGENIE_PHENO}')))

dim(pheno)
spec(pheno)

In [ ]:
pheno %>%
    group_by(sex) %>%
    summarize(count = n())

In [ ]:
pheno %>%
    group_by(statin_use) %>%
    summarize(count = n())

In [ ]:
pheno <- pheno %>%
    mutate(
        age_group_smaller_bins = cut_width(age, width = 10, boundary = 0),
        age_group = cut_width(age, width = 15, boundary = 0),
        cohort = 'AoU',
        statin_use = ifelse(statin_use == 1, TRUE, FALSE)
    )

# Plot lipids

In [ ]:
plot_vars <- function(data, xvar, yvar, fillvar, title_detail = '', log_scale = FALSE) {
    xvar_sym <- sym(xvar)
    xvar_name <- xvar
    yvar_sym <- sym(yvar)
    yvar_name <- yvar
    fillvar_sym <- sym(fillvar)
    fillvar_name <- fillvar

    options(repr.plot.width = 16, repr.plot.height = 8)
    
    p <- data %>%
        filter(!is.na(!!yvar_sym)) %>%
        ggplot(aes(x = !!xvar_sym, y = !!yvar_sym, fill = !!fillvar_sym)) +
        geom_boxplot() +
        stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 5,
                     position = position_dodge(width = 0.9), vjust = -0.8) +
        theme(
            axis.title.x=element_text(size=14),
            axis.title.y=element_text(size=14),
        ) +
        labs(title = str_glue('{yvar_name} mg/dL per person by {xvar_name} and {fillvar_name} {title_detail}'),
             caption = PLOT_SUBTITLE)

    if(log_scale) {
        p = p + scale_y_log10()
    }

    p
}

## By age group - bin size 10 [not okay to share, groups too small]

In [ ]:
for (lipid in c('LDL_mg_dl', 'TC_mg_dl', 'HDL_mg_dl', 'TG_mg_dl')) {
    print(plot_vars(data = pheno, xvar = 'age_group_smaller_bins', yvar = lipid, fillvar = 'sex'))
}

## By age group - larger bins [okay to share]

In [ ]:
for (lipid in c('LDL_mg_dl', 'TC_mg_dl', 'HDL_mg_dl', 'TG_mg_dl')) {
    print(plot_vars(data = pheno, xvar = 'age_group', yvar = lipid, fillvar = 'sex'))
}

## By statin use

In [ ]:
for (lipid in c('LDL_mg_dl', 'TC_mg_dl', 'HDL_mg_dl', 'TG_mg_dl')) {
    print(plot_vars(data = pheno, xvar = 'sex', yvar = lipid, fillvar = 'statin_use'))
}

## By statin use and adjusted

In [ ]:
for (lipid in c('LDL_adj_mg_dl', 'TC_adj_mg_dl')) {
    print(plot_vars(data = pheno, xvar = 'sex', yvar = lipid, fillvar = 'statin_use'))
}

In [ ]:
# Special case the title for this plot.
plot_vars(data = pheno, xvar = 'sex', yvar = 'TG_log_mg_dl', fillvar = 'statin_use',
          log_scale = FALSE, title_detail = '[adjusted data is in log space]')

# Plot GWAS phenotypes


In [ ]:
plot_var_histograms <- function(data, xvar, title_detail = '', log_scale = FALSE) {
    xvar_sym <- sym(xvar)
    xvar_name <- xvar

    options(repr.plot.width = 16, repr.plot.height = 10)
    
    p <- data %>%
        filter(!is.na(!!xvar_sym)) %>%
        ggplot(aes(x = !!xvar_sym)) +
        geom_histogram(bins = 30) +
        scale_y_continuous(label = comma) +
        theme(
            axis.title.x=element_text(size=14),
            axis.title.y=element_text(size=14),
        ) +
        labs(title = str_glue('{xvar_name} {title_detail}'),
             caption = PLOT_SUBTITLE)

    if(log_scale) {
        p = p + scale_y_log10()
    }

    p
}

In [ ]:
grid.arrange(
    plot_var_histograms(data = pheno, xvar = 'TC_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'TC_adj_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'TC_adj_mg_dl_resid'),
    plot_var_histograms(data = pheno, xvar = 'TC_adj_mg_dl_norm'),
    ncol = 2,
    top = 'Total cholesterol phenotype preparation'
)

In [ ]:
grid.arrange(
    plot_var_histograms(data = pheno, xvar = 'LDL_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'LDL_adj_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'LDL_adj_mg_dl_resid'),
    plot_var_histograms(data = pheno, xvar = 'LDL_adj_mg_dl_norm'),
    ncol = 2,
    top = 'Low-density lipoprotein cholesterol phenotype preparation'
)

In [ ]:
grid.arrange(
    plot_var_histograms(data = pheno, xvar = 'HDL_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'HDL_mg_dl_resid'),
    plot_var_histograms(data = pheno, xvar = 'HDL_mg_dl_norm'),
    ncol = 2,
    top = 'High-density lipoprotein cholesterol phenotype preparation'
)

In [ ]:
grid.arrange(
    plot_var_histograms(data = pheno, xvar = 'TG_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'TG_log_mg_dl'),
    plot_var_histograms(data = pheno, xvar = 'TG_log_mg_dl_resid'),
    plot_var_histograms(data = pheno, xvar = 'TG_log_mg_dl_norm'),
    ncol = 2,
    top = 'Triglyceride phenotype preparation'
)

# Load the regenie GWAS results

Bring our results into a single dataframe with a lipid type column.

In [ ]:
combined_regenie_results <- bind_rows(
    lapply(LIPIDS, function(lipid) {
        file = REGENIE_RESULTS[lipid]
        read_delim(pipe(str_glue('gsutil cat {file}')), delim = ' ') %>%
        mutate(lipid_type = lipid)
    })) %>%
    mutate(
        p_value = 10 ^ (-1 * LOG10P),
        RSID = paste0(CHROM, ':' , GENPOS, ':', ALLELE0, ':', ALLELE1)
    )

dim(combined_regenie_results)

In [ ]:
head(combined_regenie_results)

In [ ]:
combined_regenie_results %>%
    group_by(lipid_type) %>%
    summarize(
        min_p_value = min(p_value),
        max_p_value = max(p_value),
        min_LOG10P = min(LOG10P),
        max_LOG10P = max(LOG10P),
        min_A1FREQ = min(A1FREQ),
        max_A1FREQ = max(A1FREQ),
        min_N = min(N),
        max_N = max(N),
    )

# Plot regenie results

In [ ]:
plot_manhattan_and_qq <- function(regenie_results, manhattan_title, qq_title) {
    options(repr.plot.width = 10, repr.plot.height = 10)
    manhattan(regenie_results,
              chr='CHROM',
              bp='GENPOS',
              snp='ID',
              p='p_value',
              logp=TRUE,
              annotateTop = FALSE,
              ylim = c(0, 200),
              cex = 1.25,
              cex.axis = 1.25,
              cex.lab = 1.25,
              main = manhattan_title,
              sub = PLOT_SUBTITLE
             )

    qq(regenie_results$p_value,
       cex = 1.25,
       cex.axis = 1.25,
       cex.lab = 1.25,
       main = qq_title,
       sub = PLOT_SUBTITLE)
}

## All GWAS results

In [ ]:
map(LIPIDS, function(lipid) {
    regenie_results <- combined_regenie_results %>% filter(lipid_type == lipid)
    file = REGENIE_RESULTS[lipid]
    
    gc_score_A <- qchisq(median(regenie_results$p_value), 1, lower.tail=FALSE) / 0.456
    gc_score_B <- median(qchisq(1 - regenie_results$p_value, 1)) / qchisq(0.5, 1)
    
    message(str_glue('nrow: {nrow(regenie_results)} ncol: {ncol(regenie_results)} in {file}'))
    message(str_glue('GC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}'))
    message(
        regenie_results %>%
            group_by(TEST) %>%
        summarize(count = n())
    )

    plot_manhattan_and_qq(
        regenie_results,
        manhattan_title = str_glue('{basename(file)} results\nfrom {dirname(file)}'),
        qq_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\n GC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}')
    )
})

## Filter to common variants

In [ ]:
map(LIPIDS, function(lipid) {
    regenie_results <- combined_regenie_results %>% filter(lipid_type == lipid)
    file = REGENIE_RESULTS[lipid]
    
    # Use only common variants in GC and the plots.
    common_regenie_results <- regenie_results %>%
        filter(A1FREQ > 0.01 & A1FREQ < 0.99)
    
    gc_score_A <- qchisq(median(common_regenie_results$p_value), 1, lower.tail=FALSE) / 0.456
    gc_score_B <- median(qchisq(1 - common_regenie_results$p_value, 1)) / qchisq(0.5, 1)
    
    message(str_glue('nrow: {nrow(regenie_results)} n_sig: {nrow(regenie_results %>% filter(LOG10P > -log10(5e-08)))} in {file}'))
    message(str_glue('nrow: {nrow(common_regenie_results)} n_sig: {nrow(common_regenie_results %>% filter(LOG10P > -log10(5e-08)))} after filtering to common variants'))
    message(str_glue('GC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}'))
    message(
        regenie_results %>%
            group_by(TEST) %>%
        summarize(count = n())
    )

    plot_manhattan_and_qq(
        common_regenie_results,
        manhattan_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\ncommon variants only'),
        qq_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\ncommon variants only\tGC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}')
    )
})

## Prune variants in LD

In [ ]:
ld_pruned_variants  <- read_tsv(pipe(str_glue('gsutil cat {LD_PRUNED_VARIANTS}')), col_names = 'variant_id')

head(ld_pruned_variants)

In [ ]:
map(LIPIDS, function(lipid) {
    regenie_results <- combined_regenie_results %>% filter(lipid_type == lipid)
    file = REGENIE_RESULTS[lipid]
    
    # Use LD pruned results in GC and the QQ plot.
    ld_pruned_regenie_results <- regenie_results %>%
        filter(ID %in% ld_pruned_variants$variant_id)
    
    gc_score_A <- qchisq(median(ld_pruned_regenie_results$p_value), 1, lower.tail=FALSE) / 0.456
    gc_score_B <- median(qchisq(1 - ld_pruned_regenie_results$p_value, 1)) / qchisq(0.5, 1)
    
    message(str_glue('nrow: {nrow(regenie_results)} n_sig: {nrow(regenie_results %>% filter(LOG10P > -log10(5e-08)))} in {file}'))
    message(str_glue('nrow: {nrow(ld_pruned_regenie_results)} n_sig: {nrow(ld_pruned_regenie_results %>% filter(LOG10P > -log10(5e-08)))} after pruning variants in LD'))
    message(str_glue('GC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}'))
    message(
        regenie_results %>%
            group_by(TEST) %>%
        summarize(count = n())
    )

    plot_manhattan_and_qq(
        ld_pruned_regenie_results,
        manhattan_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\nVariants in LD pruned out.'),
        qq_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\nVariants in LD pruned out.\tGC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}')
    )
})

## Prune variants in LD and filter to common variants

In [ ]:
map(LIPIDS, function(lipid) {
    regenie_results <- combined_regenie_results %>% filter(lipid_type == lipid)
    file = REGENIE_RESULTS[lipid]
    
    # Use only common variants in GC and the plots.
    common_regenie_results <- regenie_results %>%
        filter(A1FREQ > 0.01 & A1FREQ < 0.99)
    
    # Use LD pruned results in GC and the QQ plot.
    ld_pruned_common_regenie_results <- common_regenie_results %>%
        filter(ID %in% ld_pruned_variants$variant_id)
    
    gc_score_A <- qchisq(median(ld_pruned_common_regenie_results$p_value), 1, lower.tail=FALSE) / 0.456
    gc_score_B <- median(qchisq(1 - ld_pruned_common_regenie_results$p_value, 1)) / qchisq(0.5, 1)
    
    message(str_glue('nrow: {nrow(regenie_results)} n_sig: {nrow(regenie_results %>% filter(LOG10P > -log10(5e-08)))} in {file}'))
    message(str_glue('nrow: {nrow(common_regenie_results)} n_sig: {nrow(common_regenie_results %>% filter(LOG10P > -log10(5e-08)))} after filtering to common variants only'))
    message(str_glue('nrow: {nrow(ld_pruned_common_regenie_results)} n_sig: {nrow(ld_pruned_common_regenie_results %>% filter(LOG10P > -log10(5e-08)))} after pruning variants in LD'))
    message(str_glue('GC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}'))
    message(
        regenie_results %>%
            group_by(TEST) %>%
        summarize(count = n())
    )

    plot_manhattan_and_qq(
        ld_pruned_common_regenie_results,
        manhattan_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\nVariants in LD pruned out and common variants only.'),
        qq_title = str_glue('{basename(file)} results\nfrom {dirname(file)}\nVariants in LD pruned out and common variants only.\tGC: {round(gc_score_A, 3)} {round(gc_score_B, 3)}')
    )
})

# Comparisons against other lipids studies 

## Comparison with UKB published GWAS summary

##### Rare coding variants in 35 genes associate with circulating lipid levels – a multi-ancestry analysis of 170,000 exomes. [Hindy et al 2021](https://www.biorxiv.org/content/10.1101/2020.12.22.423783v1.supplementary-material?versioned=true)

In [ ]:
download.file('https://www.biorxiv.org/content/biorxiv/early/2021/09/01/2020.12.22.423783/DC2/embed/media-2.xlsx?download=true', 'hindy.xlsx')

Bring the Hindy results into a single dataframe with a lipid type column.

In [ ]:
combined_hindy_results <- read_xlsx('hindy.xlsx', sheet = 'Table_S11', skip = 1, na = 'NA') %>%
    filter(Ancestry == 'Overall') %>%
    mutate(
        lipid_type = case_when(
            Trait == 'LDL_ADJ' ~ 'LDL',
            Trait == 'TOTAL_ADJ' ~ 'TC',
            TRUE ~ Trait
        )
    )

dim(combined_hindy_results)

In [ ]:
head(combined_hindy_results)

In [ ]:
map(LIPIDS, function(lipid) {
    hindy_results = combined_hindy_results %>%
        filter(lipid_type == lipid) %>%
        select(RSID, beta_Hindy=BETA_FE)

    in_common_results = inner_join(
        hindy_results,
        combined_regenie_results %>%
            filter(lipid_type == lipid) %>%
            select(RSID, beta_AoU_siloed=BETA)
    )
    
    num_hindy_results = nrow(hindy_results)
    num_in_common_results = nrow(in_common_results)
    result_cor = cor(in_common_results$beta_AoU_siloed, in_common_results$beta_Hindy)
    
    options(repr.plot.width = 10, repr.plot.height = 10)

    in_common_results %>%
    ggplot(aes(x = beta_Hindy, y = beta_AoU_siloed)) +
        geom_point(alpha = .5) +
        annotate(geom = 'text',
                 x = max(in_common_results$beta_Hindy),
                 y = min(in_common_results$beta_AoU_siloed),
                 hjust = 'right',
                 vjust = -1,
                 color = 'dark blue', 
                 size = 6,
                 label = c(str_glue('correlation: {round(result_cor, digits = 3)}\nN = {num_in_common_results}'))) +
        geom_abline() +
        theme(
            axis.title.x=element_text(size=14),
            axis.title.y=element_text(size=14),
        ) +
        labs(title = str_glue('{lipid} GWAS result comparison to {num_hindy_results}\nsignificant RSID from Hindy et al. 2021'),
             caption = PLOT_SUBTITLE)

})

## Comparison with TOPMed (Freeze8) Lipid GWAS

Whole genome sequence analysis of blood lipid levels in >66,000 individuals. [Selvaraj et al 2021](https://www.biorxiv.org/content/10.1101/2021.10.11.463514v1.supplementary-material)

In [ ]:
download.file('https://www.biorxiv.org/content/biorxiv/early/2021/10/12/2021.10.11.463514/DC1/embed/media-1.xlsx?download=true', 'selvaraj.xlsx')

Bring the Selvaraj results into a single dataframe with a lipid type column.

In [ ]:
selvaraj_tables = c(HDL = 'A4:L361', LDL = 'A363:L701', TC = 'A703:L1027', TG = 'A1029:L1318')

combined_selvaraj_results <- bind_rows(
    lapply(LIPIDS, function(lipid) {
        # Print some metadata for an eyeball check that we are associating the data with the correct lipid type.
        print(str_glue('{lipid} {selvaraj_tables[lipid]}'))
        first_row = as.integer(str_extract(selvaraj_tables[lipid], '\\d+'))
        print(read_xlsx('selvaraj.xlsx', sheet = 'Supplementary Table 3', range = str_glue('A{first_row - 1}:A{first_row}')))
        print(nrow(read_xlsx('selvaraj.xlsx', sheet = 'Supplementary Table 3', range = selvaraj_tables[lipid])))
        
        # Retrieve the data.
        read_xlsx('selvaraj.xlsx', sheet = 'Supplementary Table 3', range = selvaraj_tables[lipid]) %>%
        mutate(
            # Work around a bad entry in the data causing the p.value column to be of type character.
            p.value = as.numeric(p.value),
            RSID = paste0(CHR, ':' , POS, ':', Allele1, ':', Allele2),
            lipid_type = lipid
        )
    }))

dim(combined_selvaraj_results)

In [ ]:
head(combined_selvaraj_results)

In [ ]:
map(LIPIDS, function(lipid) {
    selvaraj_results = combined_selvaraj_results %>%
        filter(lipid_type == lipid) %>%
        select(RSID, beta_selvaraj=BETA)

    in_common_results = inner_join(
        selvaraj_results,
        combined_regenie_results %>%
            filter(lipid_type == lipid) %>%
            select(RSID, beta_AoU_siloed=BETA)
    )
    
    num_selvaraj_results = nrow(selvaraj_results)
    num_in_common_results = nrow(in_common_results)
    result_cor = cor(in_common_results$beta_AoU_siloed, in_common_results$beta_selvaraj)
    
    options(repr.plot.width = 10, repr.plot.height = 10)

    in_common_results %>%
    ggplot(aes(x = beta_selvaraj, y = beta_AoU_siloed)) +
        geom_point(alpha = .5) +
        annotate(geom = 'text',
                 x = max(in_common_results$beta_selvaraj),
                 y = min(in_common_results$beta_AoU_siloed),
                 hjust = 'right',
                 vjust = -1,
                 color = 'dark blue', 
                 size = 6,
                 label = c(str_glue('correlation: {round(result_cor, digits = 3)}\nN = {num_in_common_results}'))) +
        geom_abline() +
        theme(
            axis.title.x=element_text(size=14),
            axis.title.y=element_text(size=14),
        ) +
        labs(title = str_glue('{lipid} GWAS result comparison to {num_selvaraj_results}\nsignificant RSID from Selvaraj et al. 2021'),
             caption = PLOT_SUBTITLE)

})

# Comparison Hindy vs. Selvaraj

In [ ]:
map(LIPIDS, function(lipid) {
    hindy_results = combined_hindy_results %>%
        filter(lipid_type == lipid) %>%
        select(RSID, beta_Hindy=BETA_FE)

    selvaraj_results = combined_selvaraj_results %>%
        filter(lipid_type == lipid) %>%
        select(RSID, beta_selvaraj=BETA)

    in_common_results = inner_join(
        hindy_results,
        selvaraj_results
    )
    
    num_hindy_results = nrow(hindy_results)
    num_selvaraj_results = nrow(selvaraj_results)
    num_in_common_results = nrow(in_common_results)
    result_cor = cor(in_common_results$beta_selvaraj, in_common_results$beta_Hindy)
    
    options(repr.plot.width = 10, repr.plot.height = 10)

    in_common_results %>%
    ggplot(aes(x = beta_Hindy, y = beta_selvaraj)) +
        geom_point(alpha = .5) +
        annotate(geom = 'text',
                 x = max(in_common_results$beta_Hindy),
                 y = min(in_common_results$beta_selvaraj),
                 hjust = 'right',
                 vjust = -1,
                 color = 'dark blue', 
                 size = 6,
                 label = c(str_glue('correlation: {round(result_cor, digits = 3)}\nN = {num_in_common_results}'))) +
        geom_abline() +
        theme(
            axis.title.x=element_text(size=14),
            axis.title.y=element_text(size=14),
        ) +
        labs(title = str_glue('{lipid} GWAS result comparison between {num_hindy_results} significant RSID
from Hindy et al. 2021 and {num_selvaraj_results} significant RSID from
Selvaraj et al. 2021'),
             caption = PLOT_SUBTITLE)

})

# Provenance 

In [ ]:
devtools::session_info()